In [ ]:
import urllib.request
import pathlib
import shutil
import collections

import numpy as np
import pydicom
import matplotlib.pyplot as plt
import skimage.measure

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rai.model import load
from rai.data.images import paths_to_reduced_image_stack
from rai.mask.convert import mask_to_contours

from raicontours import cfg

In [ ]:
model = load.load_model()

In [ ]:
data_path = pathlib.Path('data')

In [ ]:
# # TODO: This can be downloaded in parallel.

# data_path.mkdir(exist_ok=True)

# data_root = 'https://github.com/RadiotherapyAI/data-tcia-deepmind/raw/61fd2525f9880c8b201758f43c773e515572be92/0522c0659'

# filenames = [f"CT-{item:03d}.dcm" for item in range(165)] + ["RS.dcm"]

# for filename in filenames:
#     urllib.request.urlretrieve(f"{data_root}/{filename}", data_path / filename)

In [ ]:
image_paths = [data_path / f"CT-{item:03d}.dcm" for item in range(165)]

In [ ]:
# image_stack = paths_to_image_stack(image_paths)

In [ ]:
# image_stack.shape

In [ ]:
# np.max(image_stack)

In [ ]:
x_grid, y_grid, image_stack = paths_to_reduced_image_stack(image_paths)

In [ ]:
image_stack.shape

In [ ]:
model_input = image_stack[None, 0:64, 32:96, 32:96]
model_input.shape

In [ ]:
model_output = model.predict(model_input)

In [ ]:
cfg["structures"]

In [ ]:
contours = {}

for structure_index, structure_name in enumerate(cfg["structures"]):
    this_structure_mask = model_output[0, ..., structure_index]
    
    contours_by_slice = []
    for z_index in range(64):
        this_slice_mask = this_structure_mask[z_index, ...]
        contours = mask_to_contours(x_grid, y_grid, this_slice_mask)

In [ ]:
def _plot_model_result(
    model_input, model_output
):
    contours_by_slice = _get_contours_by_slice(
        images=model_input,
        predictions=model_output,
    )

    x = list(range(model_input.shape[1]))
    vmin = 0.2
    vmax = 0.4

    ylim = [-np.inf, np.inf]
    xlim = [np.inf, -np.inf]

    axs = []

    for z_index, contours_by_name in contours_by_slice.items():
        fig, ax = plt.subplots()
        axs.append(ax)

        ax.pcolormesh(
            x,
            x,
            model_input[z_index, :, :],
            vmin=vmin,
            vmax=vmax,
            shading="nearest",
            cmap="gray",
        )

        for structure_name, contours in contours_by_name.items():
            for contour in contours:
                ax.plot(
                    contour[:, 1],
                    contour[:, 0],
                    label=structure_name.value,
                )

                xlim[1] = np.max([np.max(contour[:, 1]), xlim[1]])
                xlim[0] = np.min([np.min(contour[:, 1]), xlim[0]])
                ylim[0] = np.max([np.max(contour[:, 0]), ylim[0]])
                ylim[1] = np.min([np.min(contour[:, 0]), ylim[1]])

        ax.set_aspect("equal", "box")

        plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

    x_range = xlim[1] - xlim[0]
    y_range = ylim[0] - ylim[1]

    margin = 0.2

    xlim[0] -= x_range * margin
    xlim[1] += x_range * margin

    ylim[1] -= y_range * margin
    ylim[0] += y_range * margin

    for ax in axs:
        ax.set_ylim(ylim)
        ax.set_xlim(xlim)

    plt.show()

In [ ]:
def _get_contours_by_slice(
    images, predictions
):
    num_slices = images.shape[0]
    contours_by_slice = collections.defaultdict(dict)

    for i in range(num_slices):
        for structure_name in cfg["structures"]:
            k = cfg["structures"].index(structure_name)

            prediction_slice = predictions[i, :, :, k]

            if np.max(prediction_slice) < 127.5:
                continue

            contours = skimage.measure.find_contours(prediction_slice, level=127.5)
            contours_by_slice[i][structure_name] = contours

    return contours_by_slice

In [ ]:
_plot_model_result(inference_patch[0, ...], predicted_mask[0, ...])

In [ ]:
predicted_mask.shape